In [6]:
#Better
!pip install requests BeautifulSoup4 fire

     |████████████████████████████████| 81kB 2.2MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=1c52da710ea1af516ca1cb178610e5db2efd211564180e84994590c14b912b26
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire


In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys
import re
import fire
import json
import string
import matplotlib.dates as mdates
import seaborn as sns

pd.set_option("display.max.columns", None)

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#sentiment analysis package
!pip install textblob
from textblob import TextBlob

#general text pre-processor
#!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')

#tweet pre-processor 
!pip install tweet-preprocessor
import preprocessor as p

[nltk_data] Downloading package punkt to /home/enbeghan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

100 most influencial twitter users in africa

In [9]:
#Scrapping data from https://africafreak.com/100-most-influential-twitter-users-in-africa
africa_infl = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa',tag='h2')
africa_infl[:5]

['100. Jeffrey Gettleman (@gettleman)',
 '99. Africa24 Media (@a24media)',
 '98. Scapegoat (@andiMakinana)',
 '97. Africa Check (@AfricaCheck)',
 '96. James Copnall (@JamesCopnall)']

In [10]:
#Setting the data in most_infl into strings
def listToString(africa_infl):
    str1=""
    for x in most_infl:
        str1 +=x
        return str1

most_infl_100 = africa_infl
print(most_infl_100[:5])
    
    

['100. Jeffrey Gettleman (@gettleman)', '99. Africa24 Media (@a24media)', '98. Scapegoat (@andiMakinana)', '97. Africa Check (@AfricaCheck)', '96. James Copnall (@JamesCopnall)']


In [11]:
#separating numbers from the infl_fullname and handle
infl_counts = []
for infl_count in range(100):
    infl_count = most_infl_100[infl_count].split('.')
    infl_counts.append(infl_count)
print(infl_counts[:5])


#s = resp
#for i in s:
#    pattern = re.findall(r"@[\w]+",i)
#    print(pattern)

[['100', ' Jeffrey Gettleman (@gettleman)'], ['99', ' Africa24 Media (@a24media)'], ['98', ' Scapegoat (@andiMakinana)'], ['97', ' Africa Check (@AfricaCheck)'], ['96', ' James Copnall (@JamesCopnall)']]


In [12]:
#Getting numbers from the the string infl_counts
numbers = []
for number in infl_counts:
    numbers.append(number[0])
print(numbers[:5])

['100', '99', '98', '97', '96']


In [13]:
#Separating the infl_fullname from the handle
infl_names = []
for infl_name in infl_counts:
    infl_names.append(infl_name[-1].split("("))
print(infl_names[:3])

[[' Jeffrey Gettleman ', '@gettleman)'], [' Africa24 Media ', '@a24media)'], [' Scapegoat ', '@andiMakinana)']]


In [14]:
#Getting the infl_fullnme 
infl_fullname = []
for infl_name in infl_names:
    infl_fullname.append(infl_name[0].strip())
print(infl_fullname[:5])

['Jeffrey Gettleman', 'Africa24 Media', 'Scapegoat', 'Africa Check', 'James Copnall']


In [15]:
#Getting the infl_twitter handle
twitter_handle = []
for infl_name in infl_names:
    twitter_handle.append(infl_name[-1][:-1])
print(twitter_handle[:5])


['@gettleman', '@a24media', '@andiMakinana', '@AfricaCheck', '@JamesCopnall']


In [16]:
# putting the infl_fullname, his or her twitter_hand and his/her position
infl_info=(list(zip(numbers,infl_fullname,twitter_handle)))

In [17]:
#Creating a dataframe
africa_infl = pd.DataFrame(infl_info ,columns = ['Count','Name', 'User_Handle']) 
africa_infl.to_csv('africa_infl.csv')
africa_infl

,Count,Name,User_Handle
0,100,Jeffrey Gettleman,@gettleman
1,99,Africa24 Media,@a24media
2,98,Scapegoat,@andiMakinana
3,97,Africa Check,@AfricaCheck
4,96,James Copnall,@JamesCopnall
...,...,...,...
95,5,Julius Sello Malema,@Julius_S_Malema
96,4,News24,@News24
97,3,Zuma,@SAPresident
98,2,Gareth Cliff,@GarethCliff


In [18]:
#making title
africa_infl["Count"]= africa_infl.Count.astype(int)
africa_infl.set_index('Count', inplace=True)
africa_infl.sort_values('Count', inplace=True)



In [19]:
#Top 100 Africa influencers
most_influential_user = africa_infl.head(10)
most_influential_user.to_csv('most_infl_10.csv')
most_influential_user

,Name,User_Handle
Count,,
1,Trevor Noah,@Trevornoah
2,Gareth Cliff,@GarethCliff
3,Zuma,@SAPresident
4,News24,@News24
5,Julius Sello Malema,@Julius_S_Malema
6,Helen Zille,@helenzille
7,mailandguardian,@mailandguardian
8,5FM,@5FM
9,loyiso gola,@loyisogola


African Leader response to Coronavirius

In [20]:
#Scrapping data from https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa
#url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
#response = simple_get(url)

url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/'
names = get_elements(url, tag ='a', search={'class':'css-4rbku5 css-18t94o4 css-1dbjc4n r-1loqt21 r-1wbh5a2 r-dnmrzs r-1ny4l3l'})
names = get_elements(url, tag ='div', search={})
names[:30]

['Issues',
 'Regions',
 'Insights & impact',
 'People',
 'Programs',
 'Events',
 '',
 'Get involved',
 'Support the Council',
 'Sign up',
 'About the Council',
 'Media',
 'Careers',
 'Home',
 'Issues',
 'Regions',
 'Search',
 'Menu',
 '+',
 'Issues',
 'Regions',
 'Insights & impact',
 'People',
 'Programs',
 'Events',
 '',
 'Get involved',
 'Support the Council',
 'Sign up',
 'About the Council']

In [21]:
#Obtaining the specific strings with the twitter name and handle.
items = []
for elements in names:
    resp = re.findall(r'\(.*?\)', elements)
    items.append(resp)

tweet_handles = []
for i in items:
    for element in i:
        if element.startswith('(@'):
            tweet_handles.append(element.strip('(,)'))
            
# first 10 handles
tweet_handles[:20]

['@EswatiniGovern1',
 '@MalawiGovt',
 '@hagegeingob',
 '@FinanceSC',
 '@PresidencyZA',
 '@mohzambia',
 '@edmnangagwa',
 '@MinSantedj',
 '@hawelti',
 '@StateHouseKenya',
 '@PaulKagame',
 '@M_Farmaajo',
 '@SouthSudanGov',
 '@SudanPMHamdok',
 '@TZSpokesperson',
 '@KagutaMuseveni',
 '@angola_Mirex',
 '@willynyamitwe',
 '@Cherif_MZ',
 '@Presidence_RDC']

In [22]:
african_leaders = pd.DataFrame(tweet_handles,columns = ['Handle'])
african_leaders.to_csv('African_Leader_response.csv')
african_leaders

,Handle
0,@EswatiniGovern1
1,@MalawiGovt
2,@hagegeingob
3,@FinanceSC
4,@PresidencyZA
...,...
211,@IssoufouMhm
212,@MBuhari
213,@Macky_Sall
214,@PresidentBio


In [23]:
#Top 100 Africa influencers
african_leader_10 = african_leaders.head(10)
african_leader_10.to_csv('most_infl_10.csv')
african_leader_10

,Handle
0,@EswatiniGovern1
1,@MalawiGovt
2,@hagegeingob
3,@FinanceSC
4,@PresidencyZA
5,@mohzambia
6,@edmnangagwa
7,@MinSantedj
8,@hawelti
9,@StateHouseKenya


Twitter scrapping of the 100 influencers and top government officials.

In [24]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

In [3]:
consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')

In [84]:
df = pd.DataFrame(columns=['screen_name','description','number_of_tweets','following', 'followers','likes', 'retweets', 'hashtags', 'mentions'])

def get_it(account_list):
    for target in account_list: 
        item = api.get_user(target)
        name = item.name
        screen_name = item.screen_name
        description = item.description
        number_of_tweets = item.statuses_count
        following = item.friends_count
        followers = item.followers_count
        
        # age of account
        account_created_date = item.created_at
        delta = datetime.utcnow() - account_created_date
        account_age_days = delta.days
        if account_age_days > 0:
            avg_tweets_per_day = float(number_of_tweets)/float(account_age_days)


        # tweets (hashtags and mentions)
        global hashtags, mentions, replies, comments  # making them global in order to intergrate them to the df later
        hashtags = []
        mentions = []
        comments = []
        retweet_count = []
        likes_count = []
        replies = []

        tweet_count = 0
        end_date = datetime.utcnow() - timedelta(days=30)
        for status in Cursor(api.user_timeline, id=target,include_rts=False).items():                
            tweet_count += 1
            if hasattr(status, "entities"):
                entities = status.entities

    #     process_status(status)
           # tweet.entities['hashtags']
            #if hasattr(status, "hashtags"):
                #hashtag = status.hashtags
                #if hashtag is not None:
                    #hashtags.append(hashtag)
            #hashtags
            if "hashtags" in entities:
                for ent in entities["hashtags"]:
                    if ent is not None:
                        if "text" in ent:
                            hashtag = ent["text"]
                            if hashtag is not None:
                                hashtags.append(hashtag)
            
            #mentions  (will fetch other users but will later use to do mention counts between the involved users)                   
            if "user_mentions" in entities:
                for ent in entities["user_mentions"]:
                    if ent is not None:
                        if "screen_name" in ent:
                            name = ent["screen_name"]
                            #if name == target:
                            if name is not None:
                                mentions.append(name)

            # replies count                  
            if hasattr(status, "reply_count"):
                comment = status.reply_count
                if comment is not None:
                    comments.append(comment)


            # retweets count     
            if hasattr(status, "retweet_count"):
                retweets = status.retweet_count
                if retweets is not None:
                    retweet_count.append(retweets)


            # likes count     
            if hasattr(status, "favorite_count"):
                likes = status.favorite_count 
                if likes is not None:
                    likes_count.append(likes)

            # replies
            if hasattr(status, 'in_reply_to_status_id_str'):
                if (status.in_reply_to_status_id_str == status.id_str):
                    replies.append(status.text)



            # break if tweets are older than end date                 
            if status.created_at < end_date:
                break
          
            df.loc[target, ['screen_name']] = screen_name
            df.loc[target, ['description']] = description
            df.loc[target, ['number_of_tweets']] = number_of_tweets
            df.loc[target, ['following']] = following
            df.loc[target, ['followers']] = followers
            df.loc[target, ['likes']] = sum(likes_count)
            df.loc[target, ['retweets']] = sum(retweet_count)
            #df.loc[target, ['hashtags']] = hashtags
            #df.loc[target, ['mentions']] = mentions
            
     #print(hashtags)  
    return df


In [70]:
the_100 = africa_infl.User_Handle.unique()
the_leaders = african_leaders.Handle.unique()
l1 = the_100.astype(str).tolist() 
l2 = the_leaders.astype(str).tolist()
accounts = l1 + l2
#print(accounts)
# take care of private, suspended or deleted accounts
names = []
for user in accounts:
    try:
        u=api.get_user(user)
        names.append(u.screen_name)
    except Exception:
            pass
names

['Trevornoah',
 'GarethCliff',
 'SAPresident',
 'News24',
 'Julius_S_Malema',
 'helenzille',
 'mailandguardian',
 '5FM',
 'loyisogola',
 'Computicket',
 'MTVBaseAfrica',
 'OfficialCSA',
 'UlrichJvV',
 'euphonik',
 'BBCAndrewH',
 'Camfed',
 'BobSkinstad',
 'SamsungSA',
 'liveamp',
 'hartleyr',
 'SmithInAfrica',
 'gertjohan',
 '_AfricanUnion',
 'SandtonCity',
 'ParliamentofRSA',
 'Mo_IbrahimFdn',
 'NDOCKenya',
 'PeterGreste',
 'JendayiFrazer',
 'art2gee',
 'IECSouthAfrica',
 'forbesafrica',
 'Entrepreneur_SA',
 'geoffreyyork',
 'HenleyAfrica',
 'IFCAfrica',
 'africaprogress',
 'MbuyiseniNdlozi',
 'FoodBlogCT',
 'AfricaResearch',
 'MadeItInAfrica',
 'cnbcafrica',
 'willintune',
 'AfricaGoodNews',
 'ThinkAfricaFeed',
 'CityofJoburgZA',
 'TheAfricaReport',
 'schneiderhome',
 'TheStar_news',
 'rangerdiaries',
 'daniel_howden',
 'Aynte',
 'GautengProvince',
 'TechCentral',
 'africamedia_CPJ',
 'savetherhino',
 'mary_harper',
 'OkayAfrica',
 'africagathering',
 'MercedesBenz_SA',
 'ayittey',
 

In [89]:
api.verify_credentials()

User(_api=<tweepy.api.API object at 0x7f164b9f1860>, _json={'id': 2510560750, 'id_str': '2510560750', 'name': 'Enbeghan', 'screen_name': 'Enbeghan', 'location': 'kumasi', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': True, 'followers_count': 14, 'friends_count': 84, 'listed_count': 0, 'created_at': 'Sat Apr 26 13:34:31 +0000 2014', 'favourites_count': 7, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 32, 'lang': None, 'status': {'created_at': 'Wed Aug 07 17:45:56 +0000 2019', 'id': 1159158750575550464, 'id_str': '1159158750575550464', 'text': '#NewProfilePic', 'truncated': False, 'entities': {'hashtags': [{'text': 'NewProfilePic', 'indices': [0, 14]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_r

In [91]:
data = get_it(names)
data.to_csv('twitter_datap.csv')


Rate limit reached. Sleeping for: 455


In [92]:
data.head()

,screen_name,description,number_of_tweets,following,followers,likes,retweets,hashtags,mentions
Trevornoah,Trevornoah,Comedian from South Africa. I was in the crowd...,11186,325,10808846,182427,23965,NaN,NaN
GarethCliff,GarethCliff,President of https://t.co/scMZ7lsVKF ⚜. Enquir...,31626,356,1974633,2318,430,NaN,NaN
News24,News24,South Africa's premier online news resource. F...,322920,631,3578218,214805,89075,NaN,NaN
Julius_S_Malema,Julius_S_Malema,Commander in Chief of Economic Freedom Fighter...,37220,652,3129877,176119,36525,NaN,NaN
helenzille,helenzille,"GodZille, GranZille, ex-Premier, ex-Mayor of C...",72335,2303,1443391,8536,1743,NaN,NaN


List the Twitter handles of the 10 most influential government officials in Africa in order of their influence (most influential to least influential)


In [93]:
data['popularity'] = data['retweets'] + data['likes']
popular = data.sort_values(by = 'popularity', ascending = False)
popular.to_csv('popular_based_on_twitter.csv')
popular.head(10)

,screen_name,description,number_of_tweets,following,followers,likes,retweets,hashtags,mentions,popularity
MbuyiseniNdlozi,MbuyiseniNdlozi,EFF Commissar & Member of Parliament - @EFFSou...,35974,5473,1078454,363332,61807,NaN,NaN,425139
News24,News24,South Africa's premier online news resource. F...,322920,631,3578218,214805,89075,NaN,NaN,303880
Julius_S_Malema,Julius_S_Malema,Commander in Chief of Economic Freedom Fighter...,37220,652,3129877,176119,36525,NaN,NaN,212644
Trevornoah,Trevornoah,Comedian from South Africa. I was in the crowd...,11186,325,10808846,182427,23965,NaN,NaN,206392
SudanPMHamdok,SudanPMHamdok,The official account of the Prime Minister of ...,654,115,371119,96175,16109,NaN,NaN,112284
NAkufoAddo,NAkufoAddo,Official Twitter account of Nana Addo Dankwa A...,7098,352,1508619,94095,10974,NaN,NaN,105069
MBuhari,MBuhari,This is the official account of Muhammadu Buha...,4734,26,3272558,67069,13906,NaN,NaN,80975
euphonik,euphonik,LIFE | MUSIC | PROPERTY | BRANDS | CULTURE Con...,4533,65,1755567,68749,7840,NaN,NaN,76589
KagutaMuseveni,KagutaMuseveni,President of the Republic of Uganda,6645,28,1813858,49158,6803,NaN,NaN,55961
Macky_Sall,Macky_Sall,Président de la République du Sénégal 🇸🇳,2796,171,1375862,39268,6177,NaN,NaN,45445


In [94]:
data['influence_score'] = data['followers'] - data['following']
most_influential = data.sort_values(by = 'influence_score', ascending = False)
most_influential.to_csv('most_influential based on twitter.csv')
most_influential.head(10)

,screen_name,description,number_of_tweets,following,followers,likes,retweets,hashtags,mentions,popularity,influence_score
Trevornoah,Trevornoah,Comedian from South Africa. I was in the crowd...,11186,325,10808846,182427,23965,NaN,NaN,206392,10808521
News24,News24,South Africa's premier online news resource. F...,322920,631,3578218,214805,89075,NaN,NaN,303880,3577587
MBuhari,MBuhari,This is the official account of Muhammadu Buha...,4734,26,3272558,67069,13906,NaN,NaN,80975,3272532
Julius_S_Malema,Julius_S_Malema,Commander in Chief of Economic Freedom Fighter...,37220,652,3129877,176119,36525,NaN,NaN,212644,3129225
PaulKagame,PaulKagame,"President of the Republic of Rwanda, write to:...",2862,181,1984418,22918,5255,NaN,NaN,28173,1984237
GarethCliff,GarethCliff,President of https://t.co/scMZ7lsVKF ⚜. Enquir...,31626,356,1974633,2318,430,NaN,NaN,2748,1974277
KagutaMuseveni,KagutaMuseveni,President of the Republic of Uganda,6645,28,1813858,49158,6803,NaN,NaN,55961,1813830
euphonik,euphonik,LIFE | MUSIC | PROPERTY | BRANDS | CULTURE Con...,4533,65,1755567,68749,7840,NaN,NaN,76589,1755502
PresidencyZA,PresidencyZA,This is the official Twitter page of The Presi...,18881,14,1599512,20015,7044,NaN,NaN,27059,1599498
NAkufoAddo,NAkufoAddo,Official Twitter account of Nana Addo Dankwa A...,7098,352,1508619,94095,10974,NaN,NaN,105069,1508267


In [95]:
data['relevance_score'] = data['number_of_tweets'] + data['mentions']
relevance = data.sort_values(by = 'relevance_score', ascending = False)
relevance.to_csv('relevance_based_on_twitter.csv')
relevance.head(10)

,screen_name,description,number_of_tweets,following,followers,likes,retweets,hashtags,mentions,popularity,influence_score,relevance_score
Trevornoah,Trevornoah,Comedian from South Africa. I was in the crowd...,11186,325,10808846,182427,23965,NaN,NaN,206392,10808521,NaN
GarethCliff,GarethCliff,President of https://t.co/scMZ7lsVKF ⚜. Enquir...,31626,356,1974633,2318,430,NaN,NaN,2748,1974277,NaN
News24,News24,South Africa's premier online news resource. F...,322920,631,3578218,214805,89075,NaN,NaN,303880,3577587,NaN
Julius_S_Malema,Julius_S_Malema,Commander in Chief of Economic Freedom Fighter...,37220,652,3129877,176119,36525,NaN,NaN,212644,3129225,NaN
helenzille,helenzille,"GodZille, GranZille, ex-Premier, ex-Mayor of C...",72335,2303,1443391,8536,1743,NaN,NaN,10279,1441088,NaN
mailandguardian,mailandguardian,"News, comment and analysis from Africa's best ...",144309,478,1060340,5763,4229,NaN,NaN,9992,1059862,NaN
5FM,5FM,Using the power of music to bring youthful Sou...,62841,411,1164409,12590,1901,NaN,NaN,14491,1163998,NaN
loyisogola,loyisogola,2 time Emmy nominated stand up comedian from S...,5688,3948,1085294,5916,765,NaN,NaN,6681,1081346,NaN
Computicket,Computicket,Computicket - The Ticket You Can Trust,45133,1225,200279,116,45,NaN,NaN,161,199054,NaN
MTVBaseAfrica,MTVBaseAfrica,Uniting Africa through music & taking African ...,56044,109,1416120,2739,626,NaN,NaN,3365,1416011,NaN
